In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from pathlib import Path

## Importation des données d'entraînement et de test

In [2]:
DATA_PATH = Path("../input/preprocessed")
X_tr = pd.read_csv(DATA_PATH / "X_train.csv", index_col=0,parse_dates=True)
X_test = pd.read_csv(DATA_PATH / "X_test.csv", index_col=0,parse_dates=True)
Y_tr = pd.read_csv(DATA_PATH / "y_train.csv", index_col=0,parse_dates=True)

In [3]:
X_tr = np.expm1(X_tr)
X_test = np.expm1(X_test)
Y_tr = np.expm1(Y_tr)

/usr/local/lib/python3.11/dist-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in expm1
  result = func(self.values, **kwargs)


In [4]:
holed_cols = [c for c in X_tr.columns if c.startswith("holed")]
holed_cols_t = [c for c in X_test.columns if c.startswith("holed")]
clean_cols = [c for c in X_tr.columns if c not in holed_cols]
clean_cols_t = [c for c in X_test.columns if c not in holed_cols_t]

In [5]:
assert list(holed_cols) == list(Y_tr.columns)

In [6]:
X_all = pd.concat([
    X_tr,      # 20k courbes
    X_test[clean_cols_t]  # 40k courbes
], axis=1)


In [7]:
holed_cols = [c for c in X_all.columns if c.startswith("holed")]
clean_cols = [c for c in X_all.columns if c not in holed_cols]


## Interpolation Linéaire : benchmark

In [8]:
# ============= Interpolation Linéaire (Benchmark) =============

def fill_nan_with_interpolation(column):
    """
    Méthode du benchmark : interpolation linéaire
    MAE = 104
    """
    col = column.copy()
    col = col.interpolate(method='linear', limit_direction='both')
    return col


def predict_interpolation(X_test_holed):
    """
    Prédire avec interpolation linéaire (méthode benchmark)
    
    Args:
        X_test_holed: DataFrame avec les colonnes à trous de X_test
    
    Returns:
        y_pred: DataFrame avec valeurs imputées
    """
    print("Prédiction par interpolation linéaire...")
    y_pred = X_test_holed.apply(fill_nan_with_interpolation, axis=0)
    
    # Vérifier qu'il n'y a plus de NaN
    remaining_nan = y_pred.isna().sum().sum()
    if remaining_nan > 0:
        print(f" Attention : {remaining_nan} NaN restants après interpolation")
        # Fallback : forward fill puis backward fill
        y_pred = y_pred.fillna(method='ffill').fillna(method='bfill')
    
    print(f"✓ Interpolation terminée : {y_pred.shape[1]} courbes imputées")
    return y_pred


## Config

In [9]:
class Config:
    hidden_size = 256
    num_layers = 3
    dropout = 0.3
    batch_size = 32
    learning_rate = 0.0005
    num_epochs = 50
    patience = 10
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = Config()
print(f"Using device: {config.device}")

Using device: cuda


## Classes

## Fonctions d'entraînement

In [10]:
# ============= Dataset =============

class TimeSeriesDataset(Dataset):
    """Dataset pour les séries temporelles avec valeurs manquantes"""
    
    def __init__(self, X, y=None, scaler=None, fit_scaler=False):
        """
        X: DataFrame (timestamps × courbes)
        y: DataFrame avec les vraies valeurs (même format que X)
        """
        self.X = X.values.T  # Shape: (n_series, n_timesteps)
        self.y = y.values.T if y is not None else None
        
        # Normalisation
        if fit_scaler:
            self.scaler = StandardScaler()
            # Fit sur les valeurs non-NaN uniquement
            valid_values = self.X[~np.isnan(self.X)].reshape(-1, 1)
            self.scaler.fit(valid_values)
        else:
            self.scaler = scaler
            
        # Normaliser X
        self.X_scaled = self.X.copy()
        mask = ~np.isnan(self.X)
        self.X_scaled[mask] = self.scaler.transform(self.X[mask].reshape(-1, 1)).flatten()
        
        # Remplacer les NaN par 0 pour le modèle (on utilisera le mask)
        self.X_scaled = np.nan_to_num(self.X_scaled, nan=0.0)
        
        # Normaliser Y aussi !
        if self.y is not None:
            self.y_scaled = self.scaler.transform(self.y.reshape(-1, 1)).reshape(self.y.shape)
        else:
            self.y_scaled = None
        
        # Créer le masque (1 = valeur observée, 0 = manquante)
        self.mask = (~np.isnan(self.X)).astype(np.float32)

    def __len__(self):
        return self.X.shape[0]  # Nombre de séries
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.X_scaled[idx]).unsqueeze(-1)  # (timesteps, 1)
        mask = torch.FloatTensor(self.mask[idx]).unsqueeze(-1)   # (timesteps, 1)
        
        if self.y_scaled is not None:
            y = torch.FloatTensor(self.y_scaled[idx]).unsqueeze(-1)
            return x, mask, y
        return x, mask

# ============= Modèle BiLSTM =============

class BiLSTMImputer(nn.Module):
    """Modèle BiLSTM bidirectionnel pour imputation"""
    
    def __init__(self, input_size=1, hidden_size=128, num_layers=2, dropout=0.2):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # BiLSTM
        self.bilstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True,
            batch_first=True
        )
        
        # Couche de sortie
        self.fc = nn.Linear(hidden_size * 2, 1)  # *2 car bidirectionnel
        
    def forward(self, x, mask):
        """
        x: (batch, timesteps, 1)
        mask: (batch, timesteps, 1)
        """
        # Concaténer x et mask comme input
        x_masked = torch.cat([x, mask], dim=-1)  # (batch, timesteps, 2)
        
        # Passer par le BiLSTM
        lstm_out, _ = self.bilstm(x_masked)  # (batch, timesteps, hidden*2)
        
        # Prédiction
        output = self.fc(lstm_out)  # (batch, timesteps, 1)
        
        return output


# ============= Loss personnalisée =============

class MaskedMSELoss(nn.Module):
    """MSE loss qui ne calcule l'erreur que sur les valeurs manquantes"""
    
    def forward(self, pred, target, mask):
        """
        pred: prédictions (batch, timesteps, 1)
        target: vraies valeurs (batch, timesteps, 1)
        mask: masque (batch, timesteps, 1) - 1=observé, 0=manquant
        """
        # On veut prédire uniquement les valeurs manquantes
        missing_mask = (1 - mask)  # Inverser le masque
        
        # Calculer l'erreur uniquement sur les valeurs manquantes
        error = (pred - target) ** 2
        masked_error = error * missing_mask
        
        # Moyenne sur les valeurs manquantes uniquement
        n_missing = missing_mask.sum()
        if n_missing > 0:
            loss = masked_error.sum() / n_missing
        else:
            loss = torch.tensor(0.0, device=pred.device)
        
        return loss

In [11]:
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    
    for x, mask, y in tqdm(loader, desc="Training", leave=False):
        x, mask, y = x.to(device), mask.to(device), y.to(device)
        
        optimizer.zero_grad()
        
        # Forward
        pred = model(x, mask)
        loss = criterion(pred, y, mask)
        
        # Backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)

def validate(model, loader, criterion, device, scaler):
    model.eval()
    total_loss = 0
    total_mae = 0
    
    with torch.no_grad():
        for x, mask, y in loader:
            x, mask, y = x.to(device), mask.to(device), y.to(device)
            
            pred = model(x, mask)
            loss = criterion(pred, y, mask)
            
            # Calculer MAE sur les vraies valeurs (dénormalisées)
            pred_denorm = scaler.inverse_transform(pred.cpu().numpy().reshape(-1, 1))
            y_denorm = scaler.inverse_transform(y.cpu().numpy().reshape(-1, 1))
            mask_cpu = mask.cpu().numpy().reshape(-1, 1)
            
            # MAE seulement sur les valeurs manquantes
            missing_mask = (1 - mask_cpu).astype(bool).flatten()
            if missing_mask.sum() > 0:
                mae = np.abs(pred_denorm.flatten()[missing_mask] - 
                           y_denorm.flatten()[missing_mask]).mean()
                total_mae += mae
            
            total_loss += loss.item()
    
    return total_loss / len(loader), total_mae / len(loader)

def train_model(model, train_loader, val_loader, config, scaler):
    """Entraînement avec early stopping"""
    
    criterion = MaskedMSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )
    
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(config.num_epochs):
        # Train
        train_loss = train_epoch(model, train_loader, optimizer, criterion, config.device)
        
        # Validate
        val_loss, val_mae = validate(model, val_loader, criterion, config.device, scaler)
        
        # Scheduler
        scheduler.step(val_loss)
        
        print(f"Epoch {epoch+1}/{config.num_epochs}")
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss: {val_loss:.4f}, Val MAE: {val_mae:.2f}")
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            # Sauvegarder le meilleur modèle
            torch.save(model.state_dict(), 'best_bilstm_model.pt')
            print("  → New best model saved!")
        else:
            patience_counter += 1
            
        if patience_counter >= config.patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    # Charger le meilleur modèle
    model.load_state_dict(torch.load('best_bilstm_model.pt'))
    return model

## Fonctions de prédictions

In [12]:
def predict(model, X_df, scaler, config, batch_size=64):
    """Prédire les valeurs manquantes"""
    
    # Créer dataset
    dataset = TimeSeriesDataset(X_df, scaler=scaler, fit_scaler=False)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    model.eval()
    all_predictions = []
    
    with torch.no_grad():
        for x, mask in tqdm(loader, desc="Predicting"):
            x, mask = x.to(config.device), mask.to(config.device)
            
            pred = model(x, mask)
            all_predictions.append(pred.cpu().numpy())
    
    # Concatener et reshape
    predictions = np.concatenate(all_predictions, axis=0)  # (n_series, timesteps, 1)
    predictions = predictions.squeeze(-1).T  # (timesteps, n_series)
    
    # Dénormaliser
    predictions_denorm = scaler.inverse_transform(predictions.reshape(-1, 1))
    predictions_denorm = predictions_denorm.reshape(predictions.shape)
    
    # Créer DataFrame
    pred_df = pd.DataFrame(
        predictions_denorm,
        index=X_df.index,
        columns=X_df.columns
    )
    
    # Remplir uniquement les valeurs manquantes
    result = X_df.copy()
    mask = X_df.isna()
    result[mask] = pred_df[mask]
    
    return result

# ============= Fonction principale =============
def run_bilstm_imputation(X_tr, Y_tr, holed_cols, clean_cols, config):
    """
    Pipeline complet d'entraînement et prédiction
    
    Returns:
        model: modèle entraîné
        scaler: scaler utilisé
        train_loader: pour analyse
        val_loader: pour analyse
    """
    
    # Séparer train/val (80/20 sur les courbes holed)
    n_holed = len(holed_cols)
    n_train = int(0.8 * n_holed)
    
    train_cols = holed_cols[:n_train]
    val_cols = holed_cols[n_train:]
    
    print(f"Training on {len(train_cols)} series, validating on {len(val_cols)} series")
    
    # Créer datasets
    train_dataset = TimeSeriesDataset(
        X_tr[train_cols], 
        Y_tr[train_cols], 
        fit_scaler=True
    )
    
    val_dataset = TimeSeriesDataset(
        X_tr[val_cols], 
        Y_tr[val_cols], 
        scaler=train_dataset.scaler
    )
    
    # DataLoaders
    train_loader = DataLoader(
        train_dataset, 
        batch_size=config.batch_size, 
        shuffle=True
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=config.batch_size, 
        shuffle=False
    )
    
    # Créer modèle
    model = BiLSTMImputer(
        input_size=2,  # valeur + mask
        hidden_size=config.hidden_size,
        num_layers=config.num_layers,
        dropout=config.dropout
    ).to(config.device)
    
    print(f"\nModel architecture:")
    print(model)
    print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # Entraîner
    print("\nStarting training...")
    model = train_model(model, train_loader, val_loader, config, train_dataset.scaler)
    
    return model, train_dataset.scaler, train_loader, val_loader

## Pré-entraînement

In [13]:
# ============= PRÉ-ENTRAÎNEMENT =============

def analyze_missing_patterns(X_holed):
    """
    Analyser la distribution des trous dans les données réelles
    
    Returns:
        hole_size_distribution: distribution des tailles de trous
        hole_rate_distribution: distribution des taux de NA par courbe
    """
    hole_sizes = []
    hole_rates = []
    
    for col in X_holed.columns:
        series = X_holed[col]
        is_na = series.isna()
        
        # Taux de NA pour cette courbe
        na_rate = is_na.mean()
        hole_rates.append(na_rate)
        
        # Trouver les tailles de blocs consécutifs de NaN
        in_hole = False
        current_hole_size = 0
        
        for val in is_na:
            if val:  # NaN
                if not in_hole:
                    in_hole = True
                    current_hole_size = 1
                else:
                    current_hole_size += 1
            else:  # Pas NaN
                if in_hole:
                    hole_sizes.append(current_hole_size)
                    in_hole = False
                    current_hole_size = 0
        
        # Si la série se termine par un trou
        if in_hole:
            hole_sizes.append(current_hole_size)
    
    return np.array(hole_sizes), np.array(hole_rates)


def create_masked_data_realistic(X_clean, X_holed_reference, seed=42, oversample_large=True):
    """
    Créer des données masquées en imitant la distribution réelle des trous
    avec sur-échantillonnage optionnel des gros trous
    
    Args:
        X_clean: DataFrame avec courbes complètes (pour pré-entraînement)
        X_holed_reference: DataFrame avec vraies courbes à trous (pour analyser les patterns)
        seed: graine aléatoire
        oversample_large: si True, sur-échantillonner les gros trous (>30 timesteps)
    
    Returns:
        X_masked: DataFrame avec masquage réaliste
        Y_true: DataFrame avec vraies valeurs
    """
    np.random.seed(seed)

    # ----------------------------------------------------
    # 1. ANALYSE DES DONNÉES RÉELLES
    # ----------------------------------------------------
    hole_sizes, hole_rates = analyze_missing_patterns(X_holed_reference)

    # Découpage comme dans ta version
    small_holes = hole_sizes[hole_sizes <= 12]
    medium_holes = hole_sizes[(hole_sizes > 12) & (hole_sizes < 48)]
    large_holes = hole_sizes[hole_sizes >= 48]

    if oversample_large and len(large_holes) > 0:
        large_holes_repeated = np.repeat(large_holes, 5)
        hole_sizes_augmented = np.concatenate([hole_sizes, large_holes_repeated])
    else:
        hole_sizes_augmented = hole_sizes

    # Nettoyage X_clean si besoin
    if X_clean.isna().any().any():
        X_clean = (X_clean.interpolate(method='linear', axis=0)
                           .fillna(method='bfill')
                           .fillna(method='ffill'))

    X_masked = X_clean.copy()
    Y_true = X_clean.copy()

    n_rows = X_clean.shape[0]
    n_cols = X_clean.shape[1]
    columns = X_clean.columns

    created_hole_sizes = []

    print("\nCréation des masques réalistes (version optimisée)...")

    # ----------------------------------------------------
    # 2. BOUCLE SUR LES COLONNES (OPTIMISÉE)
    # ----------------------------------------------------
    for col in tqdm(columns, desc="Masking"):
        
        # NumPy array pour vitesse
        col_values = X_masked[col].values.astype(float)
        
        # Tirer un taux de NA réaliste
        target_na_rate = np.random.choice(hole_rates)
        target_na_count = int(n_rows * target_na_rate)

        if target_na_count == 0:
            continue
        
        # Masque booléen (beaucoup plus rapide que Pandas)
        mask = np.zeros(n_rows, dtype=bool)
        masked_count = 0
        attempts = 0
        max_attempts = 1000

        # ------------------------------------------------
        # Génération optimisée des blocs
        # ------------------------------------------------
        while masked_count < target_na_count and attempts < max_attempts:
            attempts += 1

            # Tirer une taille de trou depuis la distribution réaliste
            hole_size = int(np.random.choice(hole_sizes_augmented))

            # Limiter si nécessaire
            remaining = target_na_count - masked_count
            if hole_size > remaining:
                hole_size = remaining
            if hole_size <= 0:
                break
            
            # Tirer un début possible
            start = np.random.randint(0, n_rows - hole_size)
            end = start + hole_size

            # Vérifier overlap avec NumPy (ultra rapide)
            if not mask[start:end].any():
                mask[start:end] = True
                masked_count += hole_size
                created_hole_sizes.append(hole_size)

        # Appliquer le masque en une seule opération NumPy
        col_values[mask] = np.nan
        X_masked[col] = col_values

    # ----------------------------------------------------
    # 3. STATISTIQUES FINALES
    # ----------------------------------------------------
    created_hole_sizes = np.array(created_hole_sizes)

    total_values = X_clean.size
    masked_values = X_masked.isna().sum().sum()

    print(f"\n✓ Masqué {masked_values:,} valeurs sur {total_values:,} "
          f"({masked_values/total_values*100:.1f}%)")

    if len(created_hole_sizes) > 0:
        print(f"✓ Distribution créée :")
        print(f"  - Petits trous (1-12) : "
              f"{np.sum(created_hole_sizes <= 12)}")
        print(f"  - Moyens (13-47) : "
              f"{np.sum((created_hole_sizes > 12) & (created_hole_sizes < 48))}")
        print(f"  - Gros (48+) : "
              f"{np.sum(created_hole_sizes >= 48)}")
    
    return X_masked, Y_true


def pretrain_model(model, X_tr, clean_cols, holed_cols, config, n_epochs_pretrain=20):
    """
    Pré-entraîner le modèle sur les courbes complètes
    
    Args:
        model: BiLSTMImputer non entraîné
        X_tr: DataFrame complet
        clean_cols: colonnes des courbes complètes
        holed_cols: colonnes des courbes à trous (pour analyser les patterns)
        config: Config object
        n_epochs_pretrain: nombre d'epochs de pré-entraînement
    
    Returns:
        model: modèle pré-entraîné
        scaler: scaler utilisé
    """
    print("\n" + "="*60)
    print("PHASE 1 : PRÉ-ENTRAÎNEMENT sur les 20k courbes complètes")
    print("="*60)
    
    # Créer données masquées avec distribution réaliste
    X_masked, Y_true = create_masked_data_realistic(
        X_clean=X_tr[clean_cols],
        X_holed_reference=X_tr[holed_cols],  # Analyser les vrais trous
        seed=42
    )
    
    # Split train/val (90/10 car on a beaucoup de données)
    n_clean = len(clean_cols)
    n_train = int(0.9 * n_clean)
    
    train_cols_pretrain = clean_cols[:n_train]
    val_cols_pretrain = clean_cols[n_train:]
    
    print(f"\nPré-entraînement : {len(train_cols_pretrain)} train, {len(val_cols_pretrain)} val")
    
    # Créer datasets    
    train_dataset = TimeSeriesDataset(
        X_masked[train_cols_pretrain],
        Y_true[train_cols_pretrain],
        fit_scaler=True
    )
    
    val_dataset = TimeSeriesDataset(
        X_masked[val_cols_pretrain],
        Y_true[val_cols_pretrain],
        scaler=train_dataset.scaler
    )
    
    # DataLoaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=0,  # Changé de 2 à 0 pour Colab
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=0,  # Changé de 2 à 0 pour Colab
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    # Entraîner    
    print(f"\nDébut du pré-entraînement ({n_epochs_pretrain} epochs)...")
    
    # Sauvegarder config original et réduire le LR pour stabilité
    original_epochs = config.num_epochs
    original_lr = config.learning_rate
    config.num_epochs = n_epochs_pretrain
    config.learning_rate = 0.0001  # LR plus faible pour le pré-entraînement
    
    print(f"Learning rate réduit à {config.learning_rate} pour stabilité")
    
    model = train_model(model, train_loader, val_loader, config, train_dataset.scaler)
    
    # Restaurer config
    config.num_epochs = original_epochs
    config.learning_rate = original_lr
    
    print("\n✓ Pré-entraînement terminé !")
    print("Le modèle a appris les patterns généraux de consommation électrique")
    
    return model, train_dataset.scaler


def finetune_model(model, scaler, X_tr, Y_tr, holed_cols, config):
    """
    Fine-tuner le modèle pré-entraîné sur les vraies courbes à trous
    
    Args:
        model: modèle pré-entraîné
        scaler: scaler du pré-entraînement
        X_tr: DataFrame avec courbes à trous
        Y_tr: vraies valeurs
        holed_cols: colonnes des courbes à trous
        config: Config object
    
    Returns:
        model: modèle fine-tuné
    """
    print("\n" + "="*60)
    print("PHASE 2 : FINE-TUNING sur les 1000 courbes à trous réels")
    print("="*60)
    
    # Split train/val
    n_holed = len(holed_cols)
    n_train = int(0.8 * n_holed)
    
    train_cols = holed_cols[:n_train]
    val_cols = holed_cols[n_train:]
    
    print(f"Fine-tuning : {len(train_cols)} train, {len(val_cols)} val")
    
    # Créer datasets (réutiliser le scaler du pré-entraînement!)
    train_dataset = TimeSeriesDataset(
        X_tr[train_cols],
        Y_tr[train_cols],
        scaler=scaler,
        fit_scaler=False  # Important : ne pas refitter le scaler!
    )
    
    val_dataset = TimeSeriesDataset(
        X_tr[val_cols],
        Y_tr[val_cols],
        scaler=scaler,
        fit_scaler=False
    )
    
    # DataLoaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config.batch_size,
        shuffle=False
    )
    
    # Fine-tuning avec learning rate plus faible
    print(f"\nDébut du fine-tuning avec LR réduit...")

    # Réduire le learning rate pour le fine-tuning
    original_lr = config.learning_rate
    config.learning_rate = original_lr * 0.1  # 10x plus petit
    
    model = train_model(model, train_loader, val_loader, config, scaler)
    
    # Restaurer learning rate
    config.learning_rate = original_lr
    
    print("\n✓ Fine-tuning terminé !")
    
    return model

In [14]:
def train_with_pretraining(X_tr, Y_tr, holed_cols, clean_cols, config, 
                           n_epochs_pretrain=20):
    """
    Pipeline complet : pré-entraînement + fine-tuning
    
    Args:
        X_tr: DataFrame avec toutes les courbes
        Y_tr: vraies valeurs des courbes à trous
        holed_cols: colonnes des courbes à trous
        clean_cols: colonnes des courbes complètes
        config: Config object
        n_epochs_pretrain: epochs de pré-entraînement
    
    Returns:
        model: modèle entraîné
        scaler: scaler utilisé
    """
    print("\n" + "="*60)
    print("ENTRAÎNEMENT AVEC PRÉ-ENTRAÎNEMENT")
    print("="*60)
    print(f"Courbes complètes (pré-entraînement) : {len(clean_cols)}")
    print(f"Courbes à trous (fine-tuning) : {len(holed_cols)}")
    print("="*60)
    
    # Créer le modèle
    model = BiLSTMImputer(
        input_size=2,
        hidden_size=config.hidden_size,
        num_layers=config.num_layers,
        dropout=config.dropout
    ).to(config.device)
    
    print(f"\nModèle créé : {sum(p.numel() for p in model.parameters()):,} paramètres")
    
    # PHASE 1 : Pré-entraînement
    model, scaler = pretrain_model(
        model, 
        X_tr, 
        clean_cols,
        holed_cols,
        config,
        n_epochs_pretrain=n_epochs_pretrain
    )
    
    # PHASE 2 : Fine-tuning
    model = finetune_model(
        model,
        scaler,
        X_tr,
        Y_tr,
        holed_cols,
        config
    )
    
    print("\n" + "="*60)
    print("ENTRAÎNEMENT COMPLET TERMINÉ !")
    print("="*60)
    
    return model, scaler

## Entraînement

In [15]:
config = Config()

# Entraîner avec pré-entraînement
model, scaler = train_with_pretraining(
    X_tr=X_all,
    Y_tr=Y_tr,
    holed_cols=holed_cols,
    clean_cols=clean_cols,
    config=config,
    n_epochs_pretrain=15  # Ajuster selon le temps disponible
)


ENTRAÎNEMENT AVEC PRÉ-ENTRAÎNEMENT
Courbes complètes (pré-entraînement) : 57140
Courbes à trous (fine-tuning) : 999

Modèle créé : 3,686,913 paramètres

PHASE 1 : PRÉ-ENTRAÎNEMENT sur les 20k courbes complètes


/tmp/ipykernel_48/4204779929.py:82: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method='bfill')
/tmp/ipykernel_48/4204779929.py:83: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method='ffill'))



Création des masques réalistes (version optimisée)...


Masking: 100%|██████████| 57140/57140 [01:57<00:00, 487.45it/s]



✓ Masqué 7,361,866 valeurs sur 60,396,980 (12.2%)
✓ Distribution créée :
  - Petits trous (1-12) : 2254274
  - Moyens (13-47) : 25287
  - Gros (48+) : 71706

Pré-entraînement : 51426 train, 5714 val

Début du pré-entraînement (15 epochs)...
Learning rate réduit à 0.0001 pour stabilité


...

Epoch 9/50
  Train Loss: 0.1373
  Val Loss: 0.1234, Val MAE: 82.40


Epoch 10/50
  Train Loss: 0.1351
  Val Loss: 0.1237, Val MAE: 81.91


Epoch 11/50
  Train Loss: 0.1366
  Val Loss: 0.1240, Val MAE: 82.80


Epoch 12/50
  Train Loss: 0.1361
  Val Loss: 0.1238, Val MAE: 81.77


Epoch 13/50
  Train Loss: 0.1356
  Val Loss: 0.1237, Val MAE: 81.48
Early stopping at epoch 13

✓ Fine-tuning terminé !

ENTRAÎNEMENT COMPLET TERMINÉ !


## Prédiction sur le X_test

In [16]:
X_test_holed_cols = [c for c in X_test.columns if c.startswith("holed")]
X_test_imputed = predict(model, X_test[X_test_holed_cols], scaler, config)

Predicting: 100%|██████████| 16/16 [00:04<00:00,  3.83it/s]


In [17]:
X_test_imputed

,holed_1,holed_2,holed_3,holed_4,holed_5,holed_6,holed_7,holed_8,holed_9,holed_10,...,holed_991,holed_992,holed_993,holed_994,holed_995,holed_996,holed_997,holed_998,holed_999,holed_1000
Horodate,,,,,,,,,,,,,,,,,,,,,
2023-01-09 00:00:00,1061.000000,210.594376,171.684357,20.000000,223.038864,148.0,70.000000,183.000000,88.0,163.951874,...,38.0,168.000000,32.0,285.081451,445.0,262.760101,174.697006,767.0,2029.0,129.000000
2023-01-09 00:30:00,1041.000000,160.155075,99.000000,42.000000,138.000000,164.0,59.000000,170.000000,83.0,111.364815,...,27.0,82.000000,48.0,247.368912,492.0,207.000000,121.764168,773.0,1698.0,113.000000
2023-01-09 01:00:00,995.000000,133.136932,105.000000,19.000000,145.000000,93.0,119.000000,403.000000,60.0,87.745766,...,37.0,102.000000,44.0,227.160172,461.0,218.000000,96.690773,613.0,1737.0,92.391991
2023-01-09 01:30:00,998.000000,118.447319,79.737755,34.000000,270.000000,126.0,505.000000,489.000000,66.0,78.057671,...,13.0,78.000000,33.0,218.152115,491.0,104.000000,85.266655,691.0,994.0,135.000000
2023-01-09 02:00:00,950.417297,107.814857,107.000000,21.000000,309.000000,1279.0,395.000000,288.000000,46.0,74.030022,...,62.0,107.000000,36.0,214.814774,451.0,97.000000,79.555260,844.0,1044.0,102.310936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-30 22:00:00,1724.000000,97.000000,48.000000,124.858604,424.000000,632.0,125.000000,177.169586,227.0,34.000000,...,146.0,280.694275,137.0,669.000000,881.0,596.000000,147.000000,502.0,1599.0,49.613087
2023-01-30 22:30:00,1677.000000,63.000000,40.000000,122.092903,435.000000,610.0,148.375870,180.411407,259.0,26.000000,...,66.0,264.632507,155.0,909.000000,985.0,681.000000,88.000000,741.0,1887.0,78.000000
2023-01-30 23:00:00,1648.000000,60.000000,47.003956,116.816841,428.000000,535.0,166.793823,165.000000,286.0,11.781667,...,44.0,242.556473,131.0,973.000000,940.0,206.000000,65.000000,809.0,1882.0,57.000000


In [18]:
X_test_imputed.shape

(1057, 1000)

In [19]:
pd.DataFrame(X_test_imputed, 
             index=X_test.index, 
             columns=X_test_holed_cols).to_csv("predictions_bilstm.csv")

## Ensemble learning

In [20]:
def ensemble_predictions(pred_bilstm, pred_interp, alpha=0.7, 
                        handle_nan='interp'):
    """
    Combiner les prédictions du BiLSTM et de l'interpolation linéaire
    
    Args:
        pred_bilstm: prédictions du modèle BiLSTM
        pred_interp: prédictions de l'interpolation linéaire
        alpha: poids du BiLSTM (1-alpha = poids de l'interpolation)
               alpha=1.0 → 100% BiLSTM
               alpha=0.5 → moyenne 50/50
               alpha=0.0 → 100% interpolation
        handle_nan: comment gérer les NaN du BiLSTM
                    'interp' : remplacer par interpolation
                    'zero' : remplacer par 0
                    'drop' : ignorer (garder NaN)
    
    Returns:
        predictions_ensemble: DataFrame avec prédictions combinées
    """
    print(f"\n{'='*60}")
    print(f"Ensemble avec alpha={alpha:.2f}")
    print(f"  → {alpha*100:.0f}% BiLSTM + {(1-alpha)*100:.0f}% Interpolation")
    print(f"{'='*60}")
    
    # Vérifier les dimensions
    assert pred_bilstm.shape == pred_interp.shape, \
        f"Shapes incompatibles : {pred_bilstm.shape} vs {pred_interp.shape}"
    
    # Gérer les NaN du BiLSTM
    nan_count_bilstm = pred_bilstm.isna().sum().sum()
    if nan_count_bilstm > 0:
        print(f"BiLSTM contient {nan_count_bilstm} NaN")
        
        if handle_nan == 'interp':
            print(f"  → Remplacement par l'interpolation linéaire")
            pred_bilstm_clean = pred_bilstm.copy()
            mask_nan = pred_bilstm_clean.isna()
            pred_bilstm_clean[mask_nan] = pred_interp[mask_nan]
        elif handle_nan == 'zero':
            print(f"  → Remplacement par 0")
            pred_bilstm_clean = pred_bilstm.fillna(0)
        else:
            pred_bilstm_clean = pred_bilstm
    else:
        pred_bilstm_clean = pred_bilstm
        print("✓ BiLSTM ne contient pas de NaN")
    
    # Ensemble pondéré
    predictions_ensemble = alpha * pred_bilstm_clean + (1 - alpha) * pred_interp
    
    # Vérification finale
    final_nan = predictions_ensemble.isna().sum().sum()
    if final_nan > 0:
        print(f"{final_nan} NaN dans l'ensemble final")
        predictions_ensemble = predictions_ensemble.fillna(method='ffill').fillna(method='bfill')
        print("  → NaN remplacés par forward/backward fill")
    
    print(f"\n✓ Ensemble créé : {predictions_ensemble.shape}")
    
    return predictions_ensemble

In [21]:
# Prédictions
pred_bilstm = X_test_imputed
pred_interp = predict_interpolation(X_test[X_test_holed_cols])

# Ensemble avec alpha=0.7 (recommandé)
ensemble_90 = ensemble_predictions(pred_bilstm, pred_interp, alpha=0.9)
ensemble_90.to_csv("submission_best90.csv")


Prédiction par interpolation linéaire...
✓ Interpolation terminée : 1000 courbes imputées

Ensemble avec alpha=0.90
  → 90% BiLSTM + 10% Interpolation
✓ BiLSTM ne contient pas de NaN

✓ Ensemble créé : (1057, 1000)
